In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [30]:
# upload train
df=pd.read_csv('train.csv',sep=',')

In [36]:
df[1:20]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000
10,11,20,RL,70.0,11200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,129500


Для начала загрузим обучающую выборку и обработаем ее так, чтобы RandomForest'у было удобнее работать с этими данными, и чтобы он работал эффективнее.
Столбцы с небольшим (<100) числом уникальных элементов преобразуем в несколько столбцов с фиктивными переменными используя функцию get_dummies (Для кодируемого категориального признака создаются N-1 новых признаков, где N — число категорий. Каждый i-й новый признак — бинарный характеристический признак i-й категории. То есть, например, признак MSSubClass будет разделен на несколько подпризнаков (MSSubClass20, MSSubClass30) и в нужную подкатегорию ставится единичка.) Причем признаков N-1, т.к. если если во всех N-1 позиции стоят нули, то единица однозначным образом находится в этой N-той подкатегории.
Если же в столбце число уникальных элементов велико, то смысла разбивать на фиктивные переменные нет (тут вообще есть смысл разбить на промежутки, которые будут являться подкатегориями и смотреть какое значение в какой промежуток (подкатегорию) попадает и за счет этого делать какие-то выводы, но это только мысль :)). В общем в данном лучае я оставляю данные как они есть и присоединяю их к уже обработанным столбцам.

In [44]:
for i in range(1,len(list(df))):
    
    if len(df[list(df)[i]].unique())<100:
        dumm=pd.get_dummies(df[list(df)[i]]).rename(columns=lambda x: list(df)[i] + str(x))
        nu=len(list(dumm))-1
        dumm=dumm.iloc[:,0:nu]
        if i==1:
            df_f=pd.concat([df['Id'],dumm],axis=1)
        else:
            df_f=pd.concat([df_f,dumm],axis=1)
    else:
        if i==1:
            df_f=pd.concat([df['Id'],df[list(df)[i]]],axis=1)        
        else:
            df_f=pd.concat([df_f,df[list(df)[i]]],axis=1)

In [45]:
df_f[1:10]

,Id,MSSubClass20,MSSubClass30,MSSubClass40,MSSubClass45,MSSubClass50,MSSubClass60,MSSubClass70,MSSubClass75,MSSubClass80,...,SaleTypeConLI,SaleTypeConLw,SaleTypeNew,SaleTypeOth,SaleConditionAbnorml,SaleConditionAdjLand,SaleConditionAlloca,SaleConditionFamily,SaleConditionNormal,SalePrice
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,181500
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,223500
3,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,140000
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,250000
5,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,143000
6,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,307000
7,8,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,200000
8,9,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,129900
9,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,118000


Собственно то же самое проделываю с тестовой выборкой.

In [47]:
#upload test
df1=pd.read_csv('test.csv',sep=',')

for i in range(1,len(list(df1))):
    
    if len(df1[list(df1)[i]].unique())<100:
        dumm=pd.get_dummies(df1[list(df1)[i]]).rename(columns=lambda x: list(df1)[i] + str(x))
        nu=len(list(dumm))-1
        dumm=dumm.iloc[:,0:nu]
        if i==1:
            df_ff=pd.concat([df1['Id'],dumm],axis=1)
        else:
            df_ff=pd.concat([df_ff,dumm],axis=1)
    else:
        if i==1:
            df_ff=pd.concat([df1['Id'],df1[list(df1)[i]]],axis=1)        
        else:
            df_ff=pd.concat([df_ff,df1[list(df1)[i]]],axis=1)

Далее может возникнуть такая ситуация, когда количество столбцов в тестовой и обучающей выборках различны (действительно, количество получившихся подкатегорий могут не совпадать). В данном случае я в обучающей выборке оставляю лишь те столбы, которые имеются в тестовой (те столбцы, которых нет в тестовой выборке, но есть в обучающей, логично, что не повлияют на предсказанные результаты).
Пустые ячейки по умолчанию заполняю нулями (здесь тоже может быть косяк, т.к. ноль может являться существенным значением и повлиять на результат, то есть тут надо смотреть на конкретную категорию и для каждой категории, где есть пустые ячейки, подбирать хорошее значение по умолчанию (иногда этоможет быть -1))

In [52]:
df_f=pd.DataFrame(df_f,columns = list(df_ff))
df_f=df_f.fillna(0)
X=df_f.drop('Id',axis=1)
df_ff=df_ff.fillna(0)
X1=df_ff.drop('Id',axis=1)

Выполняю подбор параметров для RandomForesta с помощью функции GridSearchCV. (Совсееем капельку улучшило результат, в отличие от параметров, которые я выставила от балды, а времени ушло оочень много)

In [56]:
def Grid_Search_CV_RFR(X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import ShuffleSplit

    estimator = RandomForestRegressor()
    param_grid = { 
            "n_estimators"      : [50, 100, 500, 1000],
            "max_depth"      : [10, 20, 30, 40],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

    grid.fit(X_train, y_train)

    return grid.best_score_ , grid.best_params_

In [57]:
#rf = RandomForestRegressor(n_estimators=500,max_depth=30,oob_score=True, random_state=0)
#rf.fit(X, df['SalePrice'])
best_score, best_params = Grid_Search_CV_RFR(X, df['SalePrice'])
print "Best Score:" ,best_score
print "Best params:",best_params

Best Score: 0.8520542805400835
Best params: {'min_samples_split': 2, 'n_estimators': 50, 'bootstrap': True, 'max_depth': 40}


In [58]:
rf = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
rf.fit(X, df['SalePrice'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Тут для самопроверки смотрю, какая ошибка получается, если запихнуть в predict саму же обучающую выборку.

In [59]:
tr_pr=rf.predict(X)
tr_pr=pd.DataFrame(tr_pr)
np.mean((df['SalePrice']-tr_pr[0])**2)

143952253.62234882

Ну и тут уже делаю предсказание для тестовой выборки.

In [60]:
pr1=rf.predict(X1)
pr1=pd.DataFrame(pr1).astype(long)
pr1=pr1.rename(columns={0:'fr_pr'})

След функцией можно посмотреть какие из признаков влияют насколько слияют на предсказание данных, но эту табличку надо развернуть в красивый вид (например, там где нолики - эти колонки можно вообщбе не учитывать)
(Это на подумать на будущее :))

In [61]:
rf.feature_importances_

array([4.52249713e-04, 6.21891512e-04, 3.58483478e-06, 1.71237727e-06,
       8.39014845e-05, 5.71910941e-03, 1.15199253e-04, 2.81026197e-04,
       7.93851747e-05, 9.60473878e-06, 5.69098832e-05, 7.42339405e-05,
       0.00000000e+00, 1.69020671e-04, 6.43547526e-06, 4.37130366e-04,
       1.34751436e-04, 1.70984414e-05, 1.33278317e-03, 5.81680085e-03,
       1.84769469e-02, 6.40336799e-06, 7.12528455e-05, 7.80914847e-04,
       5.52602637e-04, 8.44359407e-06, 7.28064101e-04, 2.40856875e-04,
       2.44041442e-04, 5.01026024e-04, 3.44052464e-04, 5.53533256e-05,
       1.23944355e-06, 4.31317379e-04, 3.93307968e-04, 1.11284819e-05,
       5.14475384e-07, 8.34282799e-06, 1.35191483e-04, 9.47500617e-05,
       2.55363891e-04, 1.52681039e-03, 1.28256302e-03, 5.94546748e-05,
       5.94593589e-05, 4.63365864e-05, 2.72630356e-04, 2.02145068e-04,
       1.83034145e-06, 1.52811224e-04, 3.65974951e-04, 8.71936728e-04,
       4.40830253e-04, 1.01152886e-04, 7.39612350e-05, 7.07721741e-05,
      

Соединяю предсказание Pr1 и Id Домов из датасета и сохраняю :)

In [62]:
sb=pd.read_csv('sample_submission.csv',sep=',')
sb=pd.concat([sb,pr1],axis=1)

In [63]:
sb1=sb[['Id','fr_pr']]
sb1=sb1.rename(columns={'fr_pr':'SalePrice'})
sb1.to_csv('C:/Elizaveta/sb7.csv', index=False, header=True)